In [24]:
# Import required modules
import pandas as pd
import glob2
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Specify paths to files
master_path = r'C:\Users\MT1070\Desktop\Master Call Volume' #\pilot-extracts'
additional_files_path = r'C:\Users\MT1070\Desktop\Master Call Volume\speech-report-extracts'

# Display names of files in master folder
master_file_names = glob2.glob(master_path + "/*.xlsx")

# Display names of files in additional files folder
additional_file_names = glob2.glob(additional_files_path + "/*.xlsx")

# Remove the master Excel file from additional_file_names list if present
additional_file_names = [file for file in additional_file_names if file != r'C:\Users\MT1070\Desktop\Master Call Volume\speech-report-extracts']

# Initialize an empty data frame to store data from all files
final_sheet = pd.DataFrame()

# Iteratively read data extracts from master files and append them to the final DataFrame
data_frames = []  # Store DataFrames from master files
for file in master_file_names:
    try:
        df = pd.read_excel(file, sheet_name=None, skipfooter=2, engine='openpyxl')
        df_concat = pd.concat(df.values(), ignore_index=True, sort=False)
        data_frames.append(df_concat)
    except Exception as e:
        print(f"Error occurred while processing file: {file}")
        print(str(e))

# Merge all DataFrames in data_frames vertically
if len(data_frames) > 0:
    final_sheet = pd.concat(data_frames, axis=0, ignore_index=True)

# Reset index to use data col in final_sheet
final_sheet.reset_index(drop=True, inplace=True)

# Read df_additional data
df_additional = pd.DataFrame()
for file in additional_file_names:
    try:
        df_additional_temp = pd.read_excel(file, sheet_name=None, skipfooter=2, engine='openpyxl')
        df_additional_temp_concat = pd.concat(df_additional_temp.values(), ignore_index=True, sort=False)
        df_additional = pd.concat([df_additional, df_additional_temp_concat], axis=0, ignore_index=True)
    except Exception as e:
        print(f"Error occurred while processing additional file: {file}")
        print(str(e))

# Merge final_sheet and df_additional horizontally
if not df_additional.empty:
    final_sheet = pd.concat([final_sheet, df_additional], axis=1)

# Find the rows that contain 'date' in any cell
date_rows = final_sheet.apply(lambda x: x.astype(str).str.contains('date', case=False)).any(axis=1)

# Drop the rows that contain 'date'
final_sheet = final_sheet[~date_rows]

# Combine data into a new Excel file
master_file_path = r"C:\Users\MT1070\Desktop\Master Call Volume\SpinSci Call Volume Evaluation.xlsx"
final_sheet.to_excel(master_file_path, sheet_name='Raw Data', index=False)

c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook con

Error occurred while processing file: C:\Users\MT1070\Desktop\Master Call Volume\~$BWH_CW_Operators_Call_Volume_Report_5.23.2023.xlsx
File is not a zip file


c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\MT1070\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook con